## Gemini that can stick to response schema when responding JSONs 

https://colab.sandbox.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/controlled-generation/intro_controlled_generation.ipynb#scrollTo=tFy3H3aPgx12

In [1]:
from vertexai.generative_models import HarmBlockThreshold
from vertexai.generative_models import GenerationConfig
from vertexai.generative_models import GenerativeModel
from vertexai.generative_models import HarmCategory
from vertexai.generative_models import Part
from vertexai.generative_models import Tool
import vertexai 
import logging 
import json 
import os 

In [2]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())

In [3]:
PROJECT_ID = 'arun-genai-bb'
LOCATION = 'us-central1'
MODEL_NAME = 'gemini-1.5-pro-001'

In [4]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './../../credentials/key.json'
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [5]:
system_instruction = ['You are data engineer']

In [6]:
response_schema = {
    "type": "array",
    "items": {
        "type": "object",
        "properties": {
            "title": {
                "type": "string",
                "pattern": "^[A-Z][a-zA-Z0-9 ]{5,50}$",  # Title must start with an uppercase letter and be between 6 to 50 characters
            },
            "author": {
                "type": "string",
            },
            "genre": {
                "type": "string",
                "enum": ["Science Fiction", "Fantasy", "Mystery", "Romance", "Thriller", "Non-Fiction"],
            },
            "recommendation_date": {
                "type": "string",
                "pattern": "^\\d{4}-\\d{2}-\\d{2}$" # YYYY-MM-DD
            },
        },
        "required": ["title", "author", "genre"],
    },
}

In [9]:
model = GenerativeModel(MODEL_NAME,
                        system_instruction=system_instruction)

In [11]:
response = model.generate_content("List a few fictional books with their titles, authors, and genres and recommendation date",
    generation_config=GenerationConfig(
        response_mime_type="application/json", response_schema=response_schema
    ),
)

print(response.text)

[{"author": "Brandon Sanderson", "genre": "Fantasy", "title": "The Way of Kings", "recommendation_date": "2023-08-25"}, {"author": "James S.A. Corey", "genre": "Science Fiction", "title": "Leviathan Wakes", "recommendation_date": "2023-08-25"}, {"author": "Tamsyn Muir", "genre": "Science Fiction", "title": "Gideon the Ninth", "recommendation_date": "2023-08-25"}, {"author": "N.K. Jemisin", "genre": "Fantasy", "title": "The Fifth Season", "recommendation_date": "2023-08-25"}] 
